In [ ]:
from openai import OpenAI
import pandas as pd

APIKEY=''

#MODEL = 'gpt-4o'
MODEL = 'gpt-4o-mini'
MAX_OUTPUT_TOKENS=100
#COLLEGE = 'College of Engineering and Computer Science'
#COLLEGE = 'College of Education'
COLLEGE = 'College of Science'
PROMPT_FILE = 'prompt-short.txt'
subject_headings_after_cleanup = "subject_headings_after_cleanup_without_blanks.csv"

In [ ]:
# Read in entire Training Dataset
df = pd.read_csv(subject_headings_after_cleanup) 
print("Total Training Dataset Length : ", df.shape)

college_df = df[df['College'] == COLLEGE].reset_index(drop=True)
print("College: [" + COLLEGE + "]" + " Dataset Length: ", college_df.shape)


In [ ]:
# Initalize connection to LLM
client = OpenAI(api_key=APIKEY)

In [ ]:
# Read in subject headings and append to end of the standard prompt

with open(PROMPT_FILE, 'r') as file: 
    prompt = file.read()

college_subject_headings = ""
for row in college_df['Subject Headings'].unique():
    college_subject_headings += row + "\n"

prompt += college_subject_headings

print("Training Prompt Length : ", len(prompt))

In [ ]:
# Trial Run

ind = 14
title = college_df['Title'][ind]
abstract = college_df['Abstract'][ind]
subject_heading = college_df['Subject Headings'][ind]

query = "Please Find the subject headings for this example: \n" + title + "\n" +  abstract

message = client.chat.completions.create(model=MODEL,messages=[{"role": "system", "content": prompt}, {"role": "user", "content": query}],temperature=1, max_tokens=50, top_p=1)

In [ ]:
predicted_headings = message.choices[0].message.content.lower()
print("Title    : ", title)
print("Predicted: ", predicted_headings.split('\n'))
print("Actual   : ", subject_heading)
print("Raw Results : ", message)

print("Heading Found: ", predicted_headings.find(subject_heading) != -1)

In [ ]:
### Run for all data
#TOTAL_RUNS=10
TOTAL_RUNS=len(df.index)
count = 0
foundHeading = 0
for index, row in college_df.iterrows():
    count = count + 1
    if count > TOTAL_RUNS:
        break
    title = row['Title']
    abstract = row['Abstract']
    subject_heading = row['Subject Headings']
    query = "Please Find the subject headings for this example: \n" + title + "\n" +  abstract
    message = client.chat.completions.create(model=MODEL,messages=[{"role": "system", "content": prompt}, {"role": "user", "content": query}],temperature=1, max_tokens=50, top_p=1)
    predicted_headings = message.choices[0].message.content.lower()
    predicted_headings_list = predicted_headings.split('\n')
    if (predicted_headings.find(subject_heading) != -1):
            foundHeading = foundHeading + 1
    print(row["Title"], row["Subject Headings"], predicted_headings_list)
    print("------- Count:  %d Found: %d  Percent Found: %0.2f (%s) (%s)" % (count, foundHeading, foundHeading * 100/count, MODEL, COLLEGE))